<h1 style="color:blue"> Praktikum 7</h1>
<h3 style="color:blue">XML andmeformaat ja serialiseerimine, <br> osalausestamine ja verbiahelate analüüs, <br> märgenduskihtide visualiseerimine</h3>

**20. oktoober 2017**

**Ülesannete esitamise tähtaeg: 29. oktoober 2017 23:55**

Käesoleva praktikumi tehniliste teemade juures tutvume XML formaadis andmete lugemisega ning Pythoni mälus olevate andmete salvestamisega binaarsel kujul (ehk serialiseerimisega). Keeletöötluse teemadest tegeleme osalausete tuvastamise ja verbiahelate analüüsiga ning kõige lõpus vaatame, kuidas märgenduskihte visualiseerida selliselt, et märgendatud sõnad / fraasid oleksid värvilised, paksus kirjas, allajoonitud vms.

## XML andmeformaat

XML andmeformaat on sarnane JSON-ile: selle eesmärgiks pakkuda andmete salvestamise ja levitamise viisi, mis oleks ühtviisi loetav nii inimesele kui ka arvutile. Ajalooliselt on XML olnud standardina kasutusel siiski kauem kui JSON ning seetõttu on salvestatud paljud varasemad tekstikogud ja korpused sellesse formaati. XML-i "sugulaskeel" on HTML (mõlemad pärinevad ühisest esivanemast: [SGML standardist](https://et.wikipedia.org/wiki/%C3%9Chtlustatud_%C3%9Cldine_Markeerimise_Keel)), mis on sisuliselt veebilehekülgede "lähtekoodi" keel -- seega enamik veebis olevast (pool)tekstilisest materjalist on salvestatud just selles formaadis.

### _BeautifulSoup_

Alustuseks vaatame viise, kuidas XML-i _parsida_ ehk automaatselt analüüsida XML struktuuri ja otsida sealt mingeid alamosi. Pythonis on üheks väga heaks ja sageli kasutatavaks lahenduseks teek [_BeautifulSoup_](https://www.crummy.com/software/BeautifulSoup/). 

Kuna _BeautifulSoup_ (moodul `bs4`) on üheks `estnltk` sõltuvuseks, peaks see teil `conda` keskkonda olema juba varasemalt installitud. Kui moodul `bs4` on mingil veidral põhjusel siiski puudu, tuleks see käsurealt installida:

    conda install bs4

Käesoleva praktikumi näidetes kasutab _BeautifulSoup_ oma kõhus teeki `lxml`, nii et see tuleks installida juurde käsuga:

    conda install lxml

Järgnevas näites on XML kujul lõik ühest märgendatud ilukirjandustekstist, _BeautifulSoup_ abil parsime selle struktuuri ja kuvame "ilustatud" kujul, nii et taanded toovad XML-i struktuuri esile:

In [1]:
from bs4 import BeautifulSoup

# Näite-XML ( lõigud pärinevad Tasakaalus korpuse ilukirjanduse failist 'tanav_jutt4.tei' )
xml_content = '''
<?xml version="1.0"?>
<text> <body> <div0 type='tervikteos'> <head> Taadeldus </head>
<p> <bibl> <author> <s> Maniakkide T&auml;nav </s> </author> </bibl> </p> <p> /---/ </p>
<p> <s> " L&auml;hme Widgeti juurde , " &uuml;tlesin . </s> <s> Widget oli kursa&otilde;de , kes elas siinsamas Annelinnas . </s> <s> Tee peal &uuml;ritas Ults mulle asja selgitada . </s> </p> <p> /---/ </p>
</div0> </body> </text>
'''

# Parsime XML-sisu lxml parseri abil
soup = BeautifulSoup(xml_content,'lxml')

# Kuvame XML elementide sisud ilusti joondatult:
print(soup.prettify())

<?xml version="1.0"?>
<html>
 <body>
  <text>
   <div0 type="tervikteos">
    Taadeldus
    <p>
     <bibl>
      <author>
       <s>
        Maniakkide Tänav
       </s>
      </author>
     </bibl>
    </p>
    <p>
     /---/
    </p>
    <p>
     <s>
      " Lähme Widgeti juurde , " ütlesin .
     </s>
     <s>
      Widget oli kursaõde , kes elas siinsamas Annelinnas .
     </s>
     <s>
      Tee peal üritas Ults mulle asja selgitada .
     </s>
    </p>
    <p>
     /---/
    </p>
   </div0>
  </text>
 </body>
</html>


* **Märkus HTML olemite kohta: ** Varasemates XML versioonides polnud kodeeringu _'utf-8'_ kasutamine otseselt toetatud ning seetõttu tuli tekstide salvestamisel _'utf-8'_ sümbolid, sealhulgas ka eesti täpitähed, konverteerida nn _HTML olemiteks_. Seetõttu võimegi eelmises näites täheldada olemeid `&uuml;` (tähistab tähte _'ü'_) ja `&otilde;` (tähistab tähte _'õ'_). Kui kasutada _BeautifulSoup_'i XML (või HTML) failide parsimiseks, ei pea olemite konverteerimise pärast muretsema: konverteerimine viiakse teegi poolt läbi automaatselt, nagu on näha ka "ilustatud" väljundis;

### XML-ist otsimine

_BeautfifulSoup_ lubab otsida XML-ist, näiteks saab leida kõikide "lauseliste"-elementide sisud:

In [2]:
soup.find_all('s')

[<s> Maniakkide Tänav </s>,
 <s> " Lähme Widgeti juurde , " ütlesin . </s>,
 <s> Widget oli kursaõde , kes elas siinsamas Annelinnas . </s>,
 <s> Tee peal üritas Ults mulle asja selgitada . </s>]

Lisaks on võimalik otsida kindlate atribuutide väärtuste järgi.

In [3]:
# Leiame elemendi, mille type='tervikteos'
soup.find(type='tervikteos')

<div0 type="tervikteos">  Taadeldus 
<p> <bibl> <author> <s> Maniakkide Tänav </s> </author> </bibl> </p> <p> /---/ </p>
<p> <s> " Lähme Widgeti juurde , " ütlesin . </s> <s> Widget oli kursaõde , kes elas siinsamas Annelinnas . </s> <s> Tee peal üritas Ults mulle asja selgitada . </s> </p> <p> /---/ </p>
</div0>

In [4]:
# Leiame vastava elemendi tekstilise sisu (ilma XML elementideta)
soup.find(type='tervikteos').text

'  Taadeldus \n    Maniakkide Tänav      /---/ \n  " Lähme Widgeti juurde , " ütlesin .   Widget oli kursaõde , kes elas siinsamas Annelinnas .   Tee peal üritas Ults mulle asja selgitada .    /---/ \n'

Positiivsetel otsingutulemustel saab omakorda rakendada alamotsinguid.

Näide: kitsendame otsingut kuni saame kätte autori nime sõne kujul:

In [5]:
soup.find(type='tervikteos').author

<author> <s> Maniakkide Tänav </s> </author>

In [6]:
soup.find(type='tervikteos').author.s.text

' Maniakkide Tänav '

Näide: kitsendame, kuni saame kätte lauseliste elementide tekstilise sisu. Kõigepealt esimese "lause" korral:

In [7]:
soup.find_all('s')[0].text

' Maniakkide Tänav '

Seejärel kõigil lausetel:

In [8]:
[ s.text for s in soup.find_all('s') ]

[' Maniakkide Tänav ',
 ' " Lähme Widgeti juurde , " ütlesin . ',
 ' Widget oli kursaõde , kes elas siinsamas Annelinnas . ',
 ' Tee peal üritas Ults mulle asja selgitada . ']

### EstNLTK funktsioonid koondkorpuse XML failide lugemiseks

Tartu Ülikooli [koondkorpuse](http://www.cl.ut.ee/korpused/segakorpus/index.php?lang=et) XML formaadis tekstide sisselugemiseks on EstNLTK-s olemas ka eraldiseisvad funktsioonid. Need leiab moodulist `estnltk.teicorpus`:

  * `parse_tei_corpus(path, target=['artikkel'], encoding=None)` -- loeb ja parsib ühe XML faili (antud kataloogiteega `path`) sisu ning tagastab faili põhjal loodud `Text` objektide järjendi; 


  * `parse_tei_corpora(root, prefix='', suffix='.xml', target=['artikkel'], encoding=None)` -- loeb ja parsib kataloogist `root` faile, mille prefiks on `prefix` ja sufiks `suffix` ning tagastab failide põhjal loodud `Text` objektide järjendi.

Mõlema funktsiooni puhul: tekstilist sisu otsitakse `div` elementidest, mille `type` väärtused on kirjeldatud järjendis `target`. Vaikeväärtus `target=['artikkel']` sobib enamiku ajakirjandustekstide parsimiseks; ilukirjanduse puhul peaks sobima väärtus `target=['tervikteos']`;

Kuigi funktsioonid lubavad vaikimisi kodeeringu täpsustamata jätta (`encoding=None`), tuleks koondkorpuse XML failide lugemisel siiski kodeering alati määrata ("utf-8").

Kogu koondkorpuse XML failide automaatset konverteerimist kirjeldab [juhend](https://estnltk.github.io/estnltk/1.4.1/tutorials/tei.html) ning koondkorpus ise on saadaval [siin veebilehel](http://www.cl.ut.ee/korpused/segakorpus/index.php?lang=et) (linke järgides jõuab sealt koondkorpuse kokkupakitud XML failideni).


### Boonusülesanne. "Tõlkes kaduma läinud" lausestus? (1 p)

Mooduli `estnltk.teicorpus` funktsioonid ei ole paraku perfektsed: nende abil tekste sisse lugedes võite avastada, et tekstide lausestus pole päris selline, nagu see oli algses XML failis `<s>`-märgendite abil esile toodud. Käesolevas ülesandes uurime seda probleemi lähemalt ning leiame selle vastu ka rohtu.

Konkreetsemalt. Kataloogis 'ilu\_xml\_0' on [TÜ koondkorpuse ilukirjanduse osast](http://www.cl.ut.ee/korpused/segakorpus/eesti_ilukirjandus_1990) pärinevad `.xml` failid. Teie ülesandeks on lugeda iga tekst mällu kahel viisil:

  1. meetodite `parse_tei_corpora()` või `parse_tei_corpus()` abil; 
  2. mooduli _BeautifulSoup_ abil; 

Seejärel tuleks võrrelda mõlema versiooni lausestusi, tuvastada, millest on tingitud lausestuste erinevused, ning parandada "lausestamise" loogikat nii, et lausete arvud lõpuks klapiksid.

** Detailid: **

 * _BeautifulSoup_ mooduli abil sisseloetud failist "lausete" kättesaamiseks piisab, kui otsite faili sisust välja kõik `<s>`-märgendite vahele jäävad tekstid;


 * Pärast andmete sisselugemist meetodiga `parse_tei_corpora` (või `parse_tei_corpus`) teostage saadud `Text` objekti(de)l lausestamine ( `text.tokenize_sentences()` ) ja võrrelge tulemusi `<s>`-märgendite vahele jäävate tekstide arvuga. Tulemuseks peaksite saama, et lausete arvud erinevad; 
 
 
 * Edasi peaksite kuidagi "inspekteerima", millest tekivad lausestuste erinevused. Vägagi soovitav on selleks luua meetod, mis väljastab lause-lause haaval mõlema versiooni sisu. ( kui otsite erinevusi käsitsi, peaksite kindlasti _Notebook_-is kirjeldama, millised erinevused leidsite ); 
 
 
 * "Tekstilise sisu" poolest peaksid mõlemad versioonid olema võrdsed, seega tuleb teil lausete arvu klappima saamiseks natukene parandada loogikat, kuidas tekstilist sisu "lausestatakse" pärast andmete sisselugemist. Estnltk [sõnestamise abimaterjal](https://estnltk.github.io/estnltk/1.4.1/tutorials/text.html#tokenization) pakub sealjuures olulisi vihjeid. Muuta võib nii esimesel kui teisel viisil "lausestamist", ent teksti sisu sealjuures muuta ei tohiks;
 
 
 * Teie lahendus peaks iga tervikteksti kohta väljastama 3 arvu: kõigepealt algsed, pärast andmete kahel viisil sisselugemist saadud lausete arvud ning seejärel lõplik lausete arv, mis saadi pärast "lausestamise" loogika kohendamist;


## Osalausete tuvastamine

Osalausestaja on programm, mille abil saab pikad ja keeruka struktuuriga loomuliku keele laused jagada väiksemateks/lihtsamateks juppideks: osalauseteks. Enamasti ongi osalaused lihtlause-sarnased: sisaldavad ühte peaverbi (finiitverbi) ning sellega süntaktiliselt seotud sõnu. 

Sissejuhatuseks vaatame kõigepealt üht näidet osalausestaja kasutamise kohta:

In [9]:
from estnltk import Text

text = 'Igaüks, kes traktori eest miljon krooni lauale laob, on huvitatud sellest, et traktor '+\
       'meenutaks lisavõimaluste poolest võimalikult palju kosmoselaeva.'

# Osalausestame teksti:
text_w_clauses = Text(text).tag_clauses()

# Kõik osalaused järjendina
text_w_clauses.clause_texts

['Igaüks on huvitatud sellest,',
 ', kes traktori eest miljon krooni lauale laob,',
 'et traktor meenutaks lisavõimaluste poolest võimalikult palju kosmoselaeva.']

Automaatne osalausete tuvastaja eristab kahte liiki osalauseid:
   - tavalised osalaused, mis on tüüpiliselt üksteisest eraldatud koma ja/või sidesõnade (_ja, ning, et, sest, kuid_ vms) abil; eelmises näites on tavalised osalaused _'Igaüks on huvitatud sellest'_ ja _'et traktor meenutaks lisavõimaluste poolest võimalikult palju kosmoselaeva.'_;
   - kiillaused, mis on sisestatud mingi lause või osalause sisse ning jagavad selle kaheks tükiks\*; eelmises näites: _'kes traktori eest miljon krooni lauale laob'_ on kiillause lause _'Igaüks on huvitatud sellest'_ sees; 
     
     \* NB! Osalausestaja kiillausete käsitlus ei ühti eesti keele traditsiooniliste grammatikakirjelduste EKG II ja [EKK](http://www.eki.ee/books/ekk09/index.php?id=482&p=5&p1=3) kiillausete definitsiooniga. Milliseid süntaktilisi üksusi ja miks osalausestajas kiillauseteks nimetatakse, seda selgitab detailsemalt programmi kirjeldav [Kaalepi ja Muischneki artikkel]( http://arhiiv.rakenduslingvistika.ee/ajakirjad/index.php/aastaraamat/article/view/ERYa8.04/6)

Milleks on osalausete tuvastajat tarvis? Osalausestatud tekstide peal on võimalik uurida mitmeid keelenähtuseid, mida automaatne  süntaksianalüüs veel lahendada ei oska või mille lahendus vajab täiustamist. 

Üheks selliseks keelenähtuseks on _kollokatsioonid_ ehk sõnade koosesinemised, millega sageli kaasneb ka eraldiseisev/metafoorne tähendus (nt _'hea seisma'_, _'hambasse puhuma'_, _'õppust võtma'_). Kollokatsioonide automaatne tuvastamine nõuab kõigepealt, et piiritleme konteksti, mille seest kollokatsioonikandidaate otsime. Kui kontekstiks valida terve lause, siis liitlausete puhul jääb sõelale üksjagu "prahti" ehk siis selliseid kandidaate, mis ületavad osalausepiire. Seega saab osalausestamise tulemust aluseks võttes leida täpsemad kollokatsioonikandidaadid.

Teiseks keelenähtuseks on sõnadevahelised _rektsiooniseosed_ ehk süntaksiseosed, kus üks sõna määrab teise olemasolu ja vormi lauses. Näiteks, verb _'hoolima'_ tingib sageli seestütlevas käändes sõna esinemise lauses (_'Kas sa hoolid **temast** ?'_) ning verb _'tutvustama'_ osastavas ja alaleütlevas käändes sõnad (_'Konsul tutvustab **kuurorti** ka oma **kaasmaalastele**.'_). Ka rektsiooniseoste uurimisel aitab osalausestamine täpsemalt piiritleda konteksti, mille sees seosed võivad sõnade vahel esineda (rektsiooniseosed tüüpiliselt ei ületa osalausepiire), ning seega parandada analüüsi kvaliteeti. Verbiahelate tuvastamine, mida uurime praktikumi teises pooles, toetub sisuliselt samuti osalausepiiride märgendusele ja otsib verbiahelaid osalausete seest.

Jätkame nüüd tehnilisemal lainel. 

Kuidas leida, millistesse osalausetesse sõnad kuuluvad? Pärast automaatset  osalausestamist lisatakse iga sõna külge (ehk siis: `'words'` kihti `Text` objektil) atribuut nimega `'clause_index'`, milles tuuakse välja sõnaga seotud osalause identifikaator:

In [10]:
from estnltk import Text

text = 'Igaüks, kes traktori eest miljon krooni lauale laob, on huvitatud sellest, et traktor meenutaks '+\
       'lisavõimaluste poolest võimalikult palju kosmoselaeva.'
text_w_clauses = Text(text).tag_clauses()

In [11]:
clause_indices = [word['clause_index'] for word in text_w_clauses['words']]
clause_indices

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2]

* Oluline on meeles pidada, et osalausete identifikaatorid on unikaalsed vaid ühe lause piires. Ehk siis: kui meil on tekstis rohkem kui üks lause, siis iga lause sees alustatakse osalausete loendamist jälle 0-ist;

Kuidas uurida osalausete struktuuri (tavalised osalaused vs kiillaused)? Lisaks atribuudile `'clause_index'` lisatakse sõnade külge atribuut nimega `'clause_annotation'`, kus tuuakse välja sõnaga seotud osalausepiiri liik (kui sõnaga on seotud osalausepiir):

In [12]:
clause_boundaries = [word['clause_annotation'] if 'clause_annotation' in word else '' for word in text_w_clauses['words']]
clause_boundaries

['',
 'embedded_clause_start',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'embedded_clause_end',
 '',
 '',
 '',
 'clause_boundary',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

Seega on võimalik `zip()` meetodi abil siduda sõnad vastavate osalauseindeksite/osalausepiiri tähistega:

In [13]:
words = [word['text'] for word in text_w_clauses['words']]

# Tekstisõned ja nendega seotud osalauseidentifikaatorid
list( zip(clause_indices, words) ) 

[(0, 'Igaüks'),
 (1, ','),
 (1, 'kes'),
 (1, 'traktori'),
 (1, 'eest'),
 (1, 'miljon'),
 (1, 'krooni'),
 (1, 'lauale'),
 (1, 'laob'),
 (1, ','),
 (0, 'on'),
 (0, 'huvitatud'),
 (0, 'sellest'),
 (0, ','),
 (2, 'et'),
 (2, 'traktor'),
 (2, 'meenutaks'),
 (2, 'lisavõimaluste'),
 (2, 'poolest'),
 (2, 'võimalikult'),
 (2, 'palju'),
 (2, 'kosmoselaeva'),
 (2, '.')]

In [14]:
# Tekstisõned ja nendega seotud osalausepiiri märgendid
list( zip(words, clause_boundaries) ) 

[('Igaüks', ''),
 (',', 'embedded_clause_start'),
 ('kes', ''),
 ('traktori', ''),
 ('eest', ''),
 ('miljon', ''),
 ('krooni', ''),
 ('lauale', ''),
 ('laob', ''),
 (',', 'embedded_clause_end'),
 ('on', ''),
 ('huvitatud', ''),
 ('sellest', ''),
 (',', 'clause_boundary'),
 ('et', ''),
 ('traktor', ''),
 ('meenutaks', ''),
 ('lisavõimaluste', ''),
 ('poolest', ''),
 ('võimalikult', ''),
 ('palju', ''),
 ('kosmoselaeva', ''),
 ('.', '')]

Vajadusel vt ka osalausestajat tutvustavat abimaterjali [EstNLTK dokumentatsioonis](https://estnltk.github.io/estnltk/1.4.1/tutorials/text.html#tagging-clauses).

### Osalausestamine ja komavead

Automaatne osalausestaja eeldab vaikimisi, et ortograafiareegleid on järgitud täpselt ning komad paiknevad seal, kus nad peaksid olema. Seega, kui analüüsitavast tekstist on mõni osalauseid eraldav koma puudu, siis kannatab ka osalausestamise kvaliteet.

Siiski sisaldab osalausestaja ka eksperimentaalset režiimi, mille puhul üritatakse osalausepiire määrata ka siis, kui komad on puudu. Et seda režiimi kasutada, tuleb `Text` objekti vaikeosalausestaja kirjutada üle uue osalausestajaga (`ClauseSegmenter`), millel on sisse lülitatud komavigade-suhtes-vähetundlikkuse lipp (`ignore_missing_commas=True`). Täpsemalt käib see nii:

In [15]:
from estnltk import Text
from estnltk import ClauseSegmenter

# Loome teksti ja anname sellele kaasa komavigade-suhtes-vähetundliku osalausestaja
segmenter = ClauseSegmenter( ignore_missing_commas=True )
text = Text('Keegi teine ka siin ju kirjutas et ütles et saab ise asjadele järgi minna aga vastust seepeale ei tulnudki.', \
            clause_segmenter = segmenter)

# Osalausestame
text = text.tag_clauses()

# Kuvame teksti osalause kaupa
for clause in text.split_by('clauses'):
    print (clause.text)

Keegi teine ka siin ju kirjutas
et ütles
et saab ise asjadele järgi minna
aga vastust seepeale ei tulnudki.


### Ülesanne 1. Võimalike komavigade tuvastamine (2 p)

Looge osalausestaja abil skript, mis tuvastab ilukirjandustekstidest (kataloogis 'ilu_xml_1') võimalikke komavigu. Skript võrdleb osalausestamise tavarežiimi tulemust komavigade-suhtes-vähetundliku režiimi tulemusega ning annab teada, kui tulemused erinevad. Skript raporteerib, millises tekstis on kõige rohkem "potentsiaalseid komavigasid" (st osalausepiiride märgendid erinevad kahte režiimi kasutades kõige rohkem), ning ühtlasi väljastab leitud erinevused. Väljund peaks olema selline, et skripti kasutajal on konteksti põhjal võimalik otsustada, kas tegu on komaveaga või "valealarmiga". Selleks tuleb väljastada / märgistada sõnad, millel osalausemärgendus erines, ning lisada kontekst ulatuses _N_ sõna enne ja pärast (sh ka üle lausepiiride ulatuv kontekst). Konteksti suurus _N_ valige ise, aga see võiks olla muutujas, nii et seda saaks lihtsasti muuta.

Detaile:

* Tekstide sisselaadimiseks on tõenäoliselt kõige mugavam kasutada meetodit `parse_tei_corpora` (või `parse_tei_corpus`); Kindlasti tuleks määrata ka kodeering ('utf-8'); Sisseloetud kirjandustekstid on varustatud ka meta-andmetega, uurige `Text` objektide võtmeid;
* Erinevuste leidmiseks tuleb igast tekstist kõigepealt luua kaks osalausestatud versiooni: tavalise osalausestajaga töödeldud versioon ning "komavigade-suhtes-vähetundliku" osalausestajaga töödeldud versioon; Seejärel tuleks versioone võrrelda sõna sõna haaval ning leida osalausepiiride erinevused;
* Konteksti väljastamine: ilmselt tuleb igal sammul teada sõna positsiooni/indeksit tekstis; kui olete tuvastanud potentsiaalse komakoha, tehke kindlaks, kui suures ulatuses sõna konteksti saab kuvada (nt teksti 2. positsioonil ei saa kuvada 10 eelnevat sõna, saab ainult 2), noppige konteksti jäävad sõnad tekstist välja ja printige ekraanile;

## Verbiahelate tuvastamine

Tekstis olevate väidete _faktilisuse_ / _modaalsuse_ analüüsimisel on üks oluline samm verbidega seotud mitmesõnaliste üksuste kindlakstegemine. Näiteks võimaldab see eristada jaatavaid väiteid eitavatest (nt _'Peaminister **astub** homme tagasi'_ vs _'Peaminister eile siiski tagasi **ei astunud**'_), kindlaid väiteid võimalikest (nt _'Ta **tuleb** homme'_ vs _'Ta **võib tulla** homme'_) ning kindlaid tegevusi soovidest / plaanidest / üritamistest jms (nt 'Juku **alustas** kodutööga' vs 'Juku **kavatseb** kodutööga **alustada**', 'Juku **tahab** kodutööd **alustada**', 'Juku kinnitusel **on võimalik** kodutööd **alustada**'). Verbiahelate tuvastaja eesmärgiks ongi leida verbidega seotud mitmesõnalised üksused, millele toetudes saab üles ehitada lause _faktilisuse_ / _modaalsuse_ automaatse analüüsi.

Konkreetsemalt katab verbiahelate tuvastaja järgmisi konstruktsioone:
   * Lause/osalause peaverbid (\*\*):
       * Jaatavad verbid, v.a. _olema_ (nt 'Pidevalt **uurivad** asjade seisu ka hollandlased');
       * Ühe- ja mitmesõnalised jaatavad _olema_-konstruktsioonid (nt 'Raha **on** alati vähe', '**Oleme** sellist kino ennegi **näinud**');
       * Eitusega peaverbid: _ei/ära/pole/ega + verb_ (nt 'Helistasin korraks Carmenile, kuid ta **ei vastanud**');    
         ( \*\* "peaverbiks" nimetame siin finiitverbi või (eituse/mitmesõnalise _olema_ puhul) eituse abiverbile / _olema_-le järgnevat verbi )
   * Peaverbide laiendused:
       * _verb + infiniitverb_ ahel, kus _infiniitverbi_ ja eelneva verbi vahel on rektsiooniseos, nt verb _kutsuma_ saab võtta _ma_-verbidest argumente ('Kevadpäike **kutsub** mind **suusatama**') ning _püüdma_ saab võtta _da_-verbidest argumente ('Aita **ei püüdnudki** Leenat **mõista**');
       * _verb + nom/adv + infiniitverb_ ahel, kus mitmesõnaline üksus _verb + nimisõna/adverb_ on rektsiooniseoses sellele järgneva infiniitverbiga; näiteks, verb _otsima_ saab moodustada mitmesõnalise üksuse sõnaga _võimalust_ ning see üksus saab võtta omakorda _da_-verbe argumentideks ('Seepärast **otsisimegi** **võimalust** kusagilt mõned ilvesed **hankida**');

Kui rektsiooniseoste tuvastamisel ei teki mitmesusi (st ei tuvastata mitut erinevat viisi ühe ahela laiendamiseks), võib verbiahelate tuvastaja algoritm laiendada üht ahelat ka mitu korda ning moodustada üksjagu pika ahela, nt lausest _'Minul oleks pidanud olema õigus ise endale külalisi kutsuda'_ leitakse ahel  **oleks** => **pidanud** => **olema** => **õigus** => **kutsuda**.

Kasutusnäide:

In [16]:
from estnltk import Text

text = Text('Juku tahtis kodutööd juba ammu alustada, aga ei saanud. Juku kinnitusel on siiski võimalik kodutööd alustada.')

# Märgendame verbiahelad 
text = text.tag_verb_chains()

# Väljastame verbiahelad
text.verb_chain_texts

['tahtis alustada', 'ei saanud', 'on võimalik alustada']

Tekstist leitud verbiahelad saab kätte atribuudist `verb_chains`:

In [17]:
# Kogu informatsioon verbiahelate kohta:
text.verb_chains

[{'analysis_ids': [[0], [0]],
  'clause_index': 0,
  'end': [11, 39],
  'mood': 'indic',
  'morph': ['V_s', 'V_da'],
  'other_verbs': False,
  'pattern': ['verb', 'verb'],
  'phrase': [1, 5],
  'pol': 'POS',
  'roots': ['taht', 'alusta'],
  'start': [5, 31],
  'tense': 'imperfect',
  'voice': 'personal'},
 {'analysis_ids': [[0], [0]],
  'clause_index': 1,
  'end': [47, 54],
  'mood': 'indic',
  'morph': ['V_neg', 'V_nud'],
  'other_verbs': False,
  'pattern': ['ei', 'verb'],
  'phrase': [8, 9],
  'pol': 'NEG',
  'roots': ['ei', 'saa'],
  'start': [45, 48],
  'tense': 'imperfect',
  'voice': 'personal'},
 {'analysis_ids': [[0, 1], [0], [0]],
  'clause_index': 0,
  'end': [74, 90, 108],
  'mood': 'indic',
  'morph': ['V_b/vad', 'A_sg n', 'V_da'],
  'other_verbs': False,
  'pattern': ['ole', 'nom/adv', 'verb'],
  'phrase': [2, 4, 6],
  'pol': 'POS',
  'roots': ['ole', 'võimalik', 'alusta'],
  'start': [72, 82, 100],
  'tense': 'present',
  'voice': 'personal'}]

Nagu teiste fraasimärgenduste puhul, nii on ka verbiahelate märgendused esitatud sõnastike kujul. Käesoleva praktikumi seisukohast on kõige olulisem info järgmiste võtmete all:

* `'pol'` -- kas verbiahel on jaatav (`'POS'`) või eitav (`'NEG'`)? Võimalik ka väärtus `'??'` (määramata);
* `'roots'` -- järjend ahela sõnade algvormidega (morf analüüsi `'root'` atribuudid);
* `'mood'` -- ahela finiitverbi kõneviis. Võimalikud väärtused: `'indic'` (kindel kv), `'imper'` (käskiv ja möönev), `'condit'` (tingiv), `'quotat'` (kaudne),  või `'??'` (määramata);
* `'tense'` -- ahela finiitverbi/konstruktsiooni grammatiline aeg. Võimalikud väärtused sõltuvad kõneviisist. Kindlas kõneviisis: `'present'`, `'imperfect'`, `'perfect'`, `'pluperfect'`, käskivas kõneviisis:  `'present'` ning tingivas ja kaudses: `'present'` ja `'past'`. Lisaks võib jääda ka määramata (`'??'`);
* `'voice'` -- ahela finiitverbi tegumood. Võimalikud väärtused: `'personal'`, `'impersonal'`, `'??'`;

Erinevalt teistest fraasimärgendustest (nt ajaväljenditest, nimeüksustest) võivad verbiahelate fraasid olla katkendlikud / sisaldada lünkasid. Seetõttu pole ka verbiahelate asukohti kirjeldavad atribuudid `'start'` ja `'end'` mitte tavalised arvud, vaid arvude järjendid, milles iga järjendi liige kirjeldab ühe ahela liikme vastavalt kas siis algust või lõpp-positsiooni tekstis.

**Grammatilised seosed.** Võtme `'roots'` all olevad algvormid ei ole organiseeritud mitte nende lauses esinemise järjekorras, vaid järjekorras, mis peegeldab sõnadevahelisi grammatilisi seoseid. Esimene liige on alati finiitverb (või abiverb, kui tegemist on eitusega) ning järgnevatel positsioonidel on iga järgnev sõna eelmise süntaktiline alluv. Näide:

In [18]:
text = Text('Ja teda kutsuda ei proovitudki?')
# Väljastame verbiahelate algvormid
text.verb_chains[0]['roots']

['ei', 'proovi', 'kutsu']

Sama järjestust kasutatakse ka järjendites `'morph'` ja `'pattern'`. Erandlikult on järjendites `'start'` ja `'end'` asukohad kirjeldatud lauses esinemise järjekorras ning ka mugavusmeetod `verb_chain_texts` paneb sõnad kokku lauses esinemise järjekorras.

Detailsemat infot verbiahelate tuvastaja kohta leiab [EstNLTK abimaterjalist](https://estnltk.github.io/estnltk/1.4.1/tutorials/text.html#verb-chain-tagging).

### Ülesanne 2. Verbiahelate statistika (1,5 p)

Looge programm, mis loeb kataloogist 'ilu_xml_2' sisse ilukirjandustekstid, lisab neile verbiahelate märgenduse, analüüsib verbiahelate tunnuseid ning väljastab tabeli, kus on iga teksti kohta toodud välja:

  1. eitusega verbiahelate %;
  2. modaalverbidega _'võima'_, _'tohtima'_ ja _'pidama'_ verbiahelate % (ainult sellised ahelad, kus modaalverb paikneb alguses);
  3. olevikuajaga verbiahelate %;
  4. mingi muu verbiahelate spetsiifilise tunnuse või struktuurielemendi loendamisel saadud % (valige see tunnus ise);
  
| tunnus 1 | tunnus 2 | ...
--- | --- | --- | ---
**teos 1** | x% | y% | ...
**teos 2** | z% | ... | ...
... | ... | ... | ...

Detaile / soovitusi: 
  - tekstide sisselaadimiseks on tõenäoliselt kõige mugavam kasutada meetodit `parse_tei_corpora` (või `parse_tei_corpus`); Kindlasti tuleks määrata ka kodeering ('utf-8'); Sisseloetud `Text` objektide võtmete alt peaks kätte saama ka teose pealkirja (autor ei pruugi alati olla kättesaadav);
  - tabeli tegemisel ilmselt kõige mugavam kasutada `pandas.DataFrame`-i; tekitage algul tühi tabel, mille mõõtmed vastavad teoste ja uuritavate tunnuste arvule, ning tekstide analüüsi käigus täitke see andmetega; kui te pole veel katsetanud andmete muutmist `DataFrame`-is, siis juhatust saab [siit](http://pandas.pydata.org/pandas-docs/stable/10min.html#setting);
  - modaalverbidega _'võima'_, _'tohtima'_ või _'pidama'_ ahelate loendamine:
     * tuleks arvestada seda, et modaalverb ei pruugi alati olla ahela alguses -- kui on tegemist eitusega või liitajaga, siis võib modaalverb olla ahela teine liige; loendada tuleks ka selliseid juhte;
     * ahelas peaks modaalverbil järgnema vähemalt üks teine verb; ahelad, kus teist verbi ei järgne, jätta välja;

### Serialiseerimine (_Pickle_)

Räägime siia vahele jälle natukene tehnilist juttu. Tekstilised formaadid nagu XML ja JSON on hästi loetavad, kuid arvutil kulub nende õigele kujule viimiseks aega. Pythoni andmestruktuuride ja objektide otse-salvestamiseks leidub ka alternatiivne ja kiirem viis, mida nimetatakse _serialiseerimiseks_. Sisuliselt tähendab serialiseerimine, et salvestamisel teisendatakse mälus olevad andmed binaarsele kujule ning failist laadimisel muudetakse need tagasi Pythoni mälus olevateks andmeteks.

Pythonis vastutab serialiseerimise eest teek [_pickle_](https://docs.python.org/3/library/pickle.html). Nagu teiste andmeformaatidegi puhul, on ka serialiseerimise tulemused platvormist sõltumatud ehk serialiseerimise abil tekitatud fail peaks olema suvalisel (Pythoniga) masinal loetav. Binaarse kuju miinuseks on aga kahjuks see, et failis olevad andmed pole enam inimsilmale loetavad.

Vaikimisi saab _pickle_ hakkama paljude standardteekide klasside serialiseerimisega ning uute standardteegi objekte kasutavate _custom_ teekidega. _pickle_'i abil saab talletada nii andmestruktuure, funktsioone, klasse kui ka klassi _instance_'e.

Näitena serialiseerime _rows_ muutuja.

In [19]:
rows = [{'a':5, 'b':3}, {'c':4, 'd':1 }]

import pickle
with open('serialized_ppl.pkl','wb') as fout:
    pickle.dump(rows, fout)

Paneme tähele, et fail tuleb avada nii lugemiseks kui kirjutamiseks _binary mode_'is (`'wb'` ja `'rb'`).

Võite iseseisvalt uurida, milline on tekitatud faili `'serialized_ppl.pkl'` sisu. (Eriti loetav see pole.)

Funktsiooni `pickle.load` abil saame serialiseeritud sisu jälle Pythonisse tagasi lugeda:

In [20]:
with open('serialized_ppl.pkl','rb') as fin:
    new_rows = pickle.load(fin)
    print(new_rows)

[{'a': 5, 'b': 3}, {'c': 4, 'd': 1}]


### Ülesanne 3. Väited ja kontraväited (3 p)

Looge programm, mis eraldab kataloogi 'ilu_xml_2' ilukirjandustekstidest (verbiahelate märgenduse järgi) väitepaarid, mis on vastandlikud (millegi eitamine/jaatamine, millegi üle vaidlemine vms). Näiteks:

   * _lause 239: Kas sa **leiutad** mingi riistapuu , millega saaks maja paarsada meetrit eemale vedada ? "_   
     _lause 240: " Selliseid riistu **pole vaja leiutada** , need on juba olemas . "_
     

   * _lause 561: Kataaride vastu peetud ristisõjaga **loodeti** armastus Maria vastu **hävitada** ._  
     _lause 562: Kuid armastust **ei hävita** miski . "_
     
   
   * _lause 250: Kunagi ammu **suutsid** inimesed **lennata** tähtedele ._   
     _lause 254: Isegi Uraanile **ei suuda** me **lennata** , õnnetule Plutole lendamine oleks aga praegu praktiliselt võimatu ._
     

Tulemus ei pea olema perfektne, st tulemuste sekka võib sattuda ka väitepaare, kus ei räägita täpselt samast asjast või mille puhul pole päris selge, kas laused ikka on vastuväitelised. Aga siiski püüdke leida selline lahendus, et valede ja vaieldavate paaride arv oleks pigem väike.

Programm peaks leidma ja väljastama umbes 50 väitepaari. Väited ei pea olema erinevates lausetes, võivad olla ka samas lauses.

Lisaks tulemuste väljastamisele tuleb need salvestada _pickle_ faili nii, et hiljem oleksid iga paari kohta kättesaadavad: väitepaari mõlema liikme verbiahelad ja nende esinemislaused (vastavad _Text_ objektid). Need andmed leiavad kasutust viimases (boonus)ülesandes.

* Detaile:
   * Siin ülesandes huvitavad meid kõige rohkem verbiahelate lõpus olevad verbid. Nimetagem neid _väiteverbideks_. Näiteks lauses _"Selliseid riistu **pole vaja leiutada**."_ on väiteverbiks _**leiutada**_; 
   * Sisuliselt tuleb siin ülesandes leida iga konkreetse väiteverbi kõik esinemiskontekstid tekstis ja jagada need paaridesse selliselt, et üks paari verbiahel on väidet jaatav ja teine eitav;
   * Üks võimalik lahenduskäik:
       * looge kõigepealt väiteverbide algvormide sagedusloend üle kogu korpuse; vastandväidete otsimist võiks alustada  sagedusloendi lõpust ehk siis vähemsagedastest verbidest, kuna sagedased verbid on tavaliselt mitmetähenduslikud ja nende puhul on raskem vastandväiteid välja sõeluda; milline on sobiv sageduslävend, see peaks selguma katsetamise käigus;
       * looge iga teksti kohta sõnastik, kus võtmeteks on väiteverbid, väärtusteks aga järjendid, mille iga element kirjeldab üht väiteverbi esinemiskonteksti (nt sisaldab vastavat verbiahelat, lauset, lausenumbrit);
       * moodustage sõnastiku järgi iga väiteverbi kohta sobivad väitepaarid, kus üks verbiahel on jaatav, teine eitav; 
       * probleemsete paaride vähendamiseks võib kasutada kauguse järgi filtreerimist: jätta välja paarid, mis on üksteisest kaugemal kui _N_ lauset (sobiva _N_ leidmine jällegi katsetamise küsimus);
       * viimase sammuna kuvage leitud paarid ekraanile (koos paaride arvuga) ning ühtlasi salvestage _pickle_ abil: neid läheb tarvis ka järgmises ülesandes;

## Märgenduskihtide visualiseerimine _PrettyPrinter_-i abil

Mõnikord on tarvis saada märgendustest kiire ülevaade. Selleks oleks tore kuvada tekst selliselt, et märgendused on kuidagi spetsiaalselt tähistatud või esile tõstetud. EstNLTK-s on selleks puhuks loodud klass [PrettyPrinter](https://estnltk.github.io/estnltk/1.4.1/tutorials/prettyprinter.html#html-prettyprinter), mis võimaldab märgenduskihte visualiseerida. Otseselt pilte see klass küll ei joonista, aga ta vormistab teksti HTML-i ehk veebilehe lähtekoodina, mida saab edaspidi juba mitmesugustel viisidel kuvada.

Vaatame seda näite varal:

In [21]:
from estnltk import PrettyPrinter

text = Text('''" Kas sa leiutad mingi riistapuu , millega saaks maja paarsada meetrit eemale vedada ? "
               " Selliseid riistu pole vaja leiutada , need on juba olemas . "''')
text.tag_verb_chains()

# Loome uue "kaunistrükkija", 
#     mis värvib kihi 'verb_chains' taustavärviga 'cyan'
pp = PrettyPrinter( background='verb_chains', background_value='cyan' )

# vormistame teksti HTML kujul
html_source = pp.render(text, True)
html_source

'<!DOCTYPE html>\n<html>\n    <head>\n        <link rel="stylesheet" type="text/css" href="prettyprinter.css">\n        <meta charset="utf-8">\n        <title>PrettyPrinter</title>\n    </head>\n    <style>\n\n\n\t\tmark {\n\t\t\tbackground:none;\n\t\t}\n\t\tmark.background {\n\t\t\tbackground-color: cyan;\n\t\t}\n\n    </style>\n    <body>\n\n&quot; Kas sa <mark class="background">leiutad</mark> mingi riistapuu , millega <mark class="background">saaks</mark> maja paarsada meetrit eemale <mark class="background">vedada</mark> ? &quot;<br/>               &quot; Selliseid riistu <mark class="background">pole</mark> <mark class="background">vaja</mark> <mark class="background">leiutada</mark> , need <mark class="background">on</mark> juba <mark class="background">olemas</mark> . &quot;\n\t</body>\n</html>'

Üks viis näha vormistatud teksti on salvestada nüüd `html_source` sisu veebilehekülje faili (ehk `.html` lõpuga tekstifaili) ning avada see fail veebibrauseris.

Aga _Notebook_ oskab ka ise HTML sisu kuvada. Seda saab teha funktsioonide `HTML` ja `display` abil:

In [22]:
from IPython.display import HTML, display
# kuvame HTML sisu Notebook'is
display(HTML(html_source))

Detailsema visualiseerimisvõimaluste kirjelduse leiate [PrettyPrinter-i abimaterjalist](https://estnltk.github.io/estnltk/1.4.1/tutorials/prettyprinter.html#html-prettyprinter). Siiski on **üks oluline asi, mida seal aga öeldud pole.** _PrettyPrinter_ töötab ainult selliste märgenduskihtide peal, kus iga üksikmärgendus on sõnastiku kujul ning sõnastiku atribuutides `'start'` ja `'end'` on toodud märgenduse asukoht ning atribuudis `'text'` vastav tekstisisu. Seetõttu ei tööta ka abimaterjalides olev sõnaliikide (POS-tagide) värvimise näide (Example #3) -- sõnaliikide kiht pole eraldiseisev kiht, vaid on morf analüüsi kihi sees ja nii sügavale _PrettyPrinter_ ei vaata.  

Selleks, et sõnaliikide järgi värvimine korrektselt töötaks, tuleb värvitava sõnaliigi (või sõnaliikide) kohta tekitada _Text_ objekti sisse uus kiht, mille liikmed on nõutud kujul. Lihtne näide:

In [23]:
text = Text('''Kas sa leiutad mingi riistapuu?''')
text.tag_analysis()

# Tekitame eraldiseisva verbide märgenduskihi
text['verbs'] = [ {'end': 14, 'start': 7, 'text': 'leiutad'} ]

# vormistame teksti HTML kujul, märgendame 'verbs' kihi
pp = PrettyPrinter( background='verbs', background_value='cyan' )
html_source = pp.render(text, True)

# kuvame
display(HTML(html_source))

### Boonusülesanne. Vastandväidete visualiseerimine (0,75 p)

Laadige eelmise ülesande tulemused _pickle_ abil uuesti mällu ja visualiseerige _PrettyPrinter_-i abil väitepaaride verbiahelad -- selliselt, et eitused on kujundatud ühte moodi ning jaatused teist moodi.

Selleks, et visualiseerimine tööle saada, tuleb visualiseeritavatele _Text_ objektidele tekitada uued märgenduskihid: üks eitavate ja teine jaatavate verbiahelate jaoks.